In [17]:
from ortools.sat.python import cp_model
import pandas as pd
import math
import streamlit as st

In [18]:
num_scheduled = {
    0: 4,
    1: 2,
    2: 3, 
    3: 4,
    4: 4,
    5: 4,
    6: 2,
    7: 4,
    8: 4
}


In [19]:
#Requests for unavailability
unavailable = [[0,3],
[0,4],
[0,12],
[0,22],
[0,23],
[0,24],
[0,25],
[0,26],
[0,29],
[0,30],
[0,37],
[0,39],
[0,47],
[0,51],
[1,1],
[1,2],
[1,3],
[1,4],
[1,5],
[1,6],
[1,7],
[1,8],
[1,9],
[1,10],
[1,11],
[1,12],
[1,13],
[1,14],
[1,15],
[1,16],
[1,17],
[1,18],
[1,19],
[1,20],
[1,21],
[1,22],
[1,23],
[1,24],
[1,25],
[1,26],
[1,27],
[1,28],
[1,29],
[1,30],
[1,31],
[1,32],
[1,33],
[1,34],
[1,37],
[1,41],
[1,44],
[1,45],
[1,46],
[1,47],
[1,48],
[1,49],
[1,50],
[1,51],
[1,52],
[2,2],
[2,4],
[2,8],
[2,9],
[2,10],
[2,11],
[2,12],
[2,13],
[2,14],
[2,16],
[2,19],
[2,20],
[2,21],
[2,22],
[2,23],
[2,26],
[2,28],
[2,33],
[2,35],
[2,37],
[2,38],
[2,39],
[2,43],
[2,44],
[2,47],
[2,51],
[2,52],
[3,4],
[3,8],
[3,13],
[3,17],
[3,22],
[3,27],
[3,28],
[3,29],
[3,30],
[3,35],
[3,39],
[3,43],
[3,44],
[3,48],
[4,14],
[4,15],
[4,16],
[4,17],
[4,18],
[4,19],
[4,20],
[4,21],
[4,26],
[4,27],
[4,28],
[4,31],
[4,32],
[4,33],
[4,34],
[4,38],
[4,39],
[4,51],
[4,52],
[5,23],
[5,24],
[5,25],
[5,26],
[5,27],
[5,28],
[5,29],
[5,30],
[5,31],
[5,32],
[5,33],
[5,34],
[5,51],
[6,12],
[6,13],
[6,21],
[6,27],
[6,31],
[6,41],
[6,42],
[6,43],
[6,47],
[6,51],
[6,52],
[7,2],
[7,4],
[7,5],
[7,6],
[7,7],
[7,8],
[7,20],
[7,21],
[7,26],
[7,27],
[7,30],
[7,34],
[7,35],
[7,41],
[7,47],
[7,48],
[7,49],
[7,50],
[7,51],
[7,52],
[8,2],
[8,3],
[8,4],
[8,5],
[8,6],
[8,7],
[8,8],
[8,9],
[8,10],
[8,11],
[8,12],
[8,13],
[8,15],
[8,18],
[8,20],
[8,21],
[8,22],
[8,23],
[8,24],
[8,25],
[8,26],
[8,27],
[8,28],
[8,29],
[8,30],
[8,34],
[8,35],
[8,36],
[8,40],
[8,41],
[8,42],
[8,43],
[8,44],
[8,45]
]


In [50]:
churches = range(9)
model = cp_model.CpModel()
num_weeks = 52 # Number of weeks being considered for the schedule
max_consecutive_gap = 1  # Maximum number of consecutive weeks without a scheduled church
scheduled_times = {}
min_gap_church = 8  # Minimum number of weeks between scheduled times for each church

In [51]:
# Create scheduled times variables and add constraints to ensure they are different
for c in churches:
    for i in range(num_scheduled[c]):  # Use the specified number of times for each church
        scheduled_times[c, i] = model.NewIntVar(0, num_weeks, f'church_{c}_for_the_{i}_time')
        if i > 0:
            model.Add(scheduled_times[c, i - 1] < scheduled_times[c, i])


In [52]:
#No times equal to eachother.
for c1, t1 in scheduled_times:
    for c2, t2 in scheduled_times:
        if c1 != c2: #and t1 > 0 and t2 > 0: 
            model.Add(scheduled_times[c1, t1] != scheduled_times[c2, t2])

In [53]:
#unavailable weeks
for c, w in unavailable:
    for t in range(num_scheduled[c]):
        model.Add(scheduled_times[c, t] != w)

In [54]:
#n weeks apart constraint

for c in churches:
    for t in range(num_scheduled[c]):
        if t>0:
            model.Add(scheduled_times[c, t-1]+ min_gap_church <= scheduled_times[c, t])

In [55]:
# Create a binary variable for each week indicating if it's scheduled or not
week_scheduled = {w: model.NewBoolVar(f'week_{w}_scheduled') for w in range(num_weeks)}

# Add constraints to link the week_scheduled variables with the scheduled_times variables
for w in range(num_weeks):
    # Create a list of boolean variables, each indicating if a particular church is scheduled in week w
    church_scheduled_this_week = []
    for c in churches:
        for t in range(num_scheduled[c]):
            church_scheduled = model.NewBoolVar(f'church_{c}_scheduled_in_week_{w}')
            model.Add(scheduled_times[c, t] == w).OnlyEnforceIf(church_scheduled)
            model.Add(scheduled_times[c, t] != w).OnlyEnforceIf(church_scheduled.Not())
            church_scheduled_this_week.append(church_scheduled)

    # If any church is scheduled in week w, set week_scheduled[w] to true
    model.AddMaxEquality(week_scheduled[w], church_scheduled_this_week)

In [56]:
# Use a sliding window of size max_consecutive_gap + 1
for w in range(num_weeks - max_consecutive_gap):
    # Count the number of unscheduled weeks in the window
    unscheduled_count = sum(week_scheduled[w + offset].Not() for offset in range(max_consecutive_gap + 1))
    # Ensure the count does not exceed max_consecutive_gap
    model.Add(unscheduled_count <= max_consecutive_gap)

In [57]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = False

In [58]:
solver.Solve(model)
print(solver.StatusName())

OPTIMAL


In [59]:
#Pretty printing
if solver.StatusName() == 'OPTIMAL':
    for c in churches:
        #print(f"Assignments for Church {c}:")
        for t in range(num_scheduled[c]):
            print(f"Church: {c} Assigned_on_week {solver.Value(scheduled_times[c,t])}")

else:
    print("No solution found or an error occurred.")

Church: 0 Assigned_on_week 0
Church: 0 Assigned_on_week 28
Church: 0 Assigned_on_week 44
Church: 0 Assigned_on_week 52
Church: 1 Assigned_on_week 35
Church: 1 Assigned_on_week 43
Church: 2 Assigned_on_week 7
Church: 2 Assigned_on_week 18
Church: 2 Assigned_on_week 50
Church: 3 Assigned_on_week 16
Church: 3 Assigned_on_week 26
Church: 3 Assigned_on_week 34
Church: 3 Assigned_on_week 42
Church: 4 Assigned_on_week 3
Church: 4 Assigned_on_week 11
Church: 4 Assigned_on_week 25
Church: 4 Assigned_on_week 36
Church: 5 Assigned_on_week 5
Church: 5 Assigned_on_week 13
Church: 5 Assigned_on_week 22
Church: 5 Assigned_on_week 40
Church: 6 Assigned_on_week 20
Church: 6 Assigned_on_week 30
Church: 7 Assigned_on_week 9
Church: 7 Assigned_on_week 23
Church: 7 Assigned_on_week 38
Church: 7 Assigned_on_week 46
Church: 8 Assigned_on_week 1
Church: 8 Assigned_on_week 14
Church: 8 Assigned_on_week 32
Church: 8 Assigned_on_week 48


In [60]:
#Print for debugging
if solver.StatusName() == 'OPTIMAL':
    for c in churches:
        for t in range(num_scheduled[c]):
            print(solver.Value(scheduled_times[c,t]))

else:
    print("No solution found or an error occurred.")

0
28
44
52
35
43
7
18
50
16
26
34
42
3
11
25
36
5
13
22
40
20
30
9
23
38
46
1
14
32
48


In [31]:
for week in range(num_weeks):
    print(solver.Value(week_scheduled[week]))

1
1
0
1
0
1
0
1
0
1
0
1
0
1
1
0
1
0
1
0
1
0
1
1
0
1
1
0
1
0
1
0
1
0
1
1
1
0
1
0
1
0
1
1
1
0
1
0
1
0
1
0


In [64]:
schedule = pd.DataFrame(columns = ['Church', 'Week'])
#Populate dataframes for printing schedule
  
if solver.StatusName() == 'OPTIMAL':
    print('A solution was found!')
    for c in churches:
        #print
        for t in range(num_scheduled[c]):
        
            #row = (columns = ['Church', 'Week'])

            schedule = pd.concat([schedule, pd.DataFrame([{'Church':c, 'Week':solver.Value(scheduled_times[c,t])}])], ignore_index = True)


A solution was found!


In [65]:
schedule

,Church,Week
0,0,0
1,0,28
2,0,44
3,0,52
4,1,35
5,1,43
6,2,7
7,2,18
8,2,50
9,3,16


In [3]:
Blackoutdates = pd.read_csv('C:\\Users\\ryder\\Documents\\Software\\ChurchesOptimization\\BlackoutDates.csv')

In [4]:
Blackoutdates

,Church,Week
0,0,3
1,0,4
2,0,12
3,0,22
4,0,23
...,...,...
192,8,41
193,8,42
194,8,43
195,8,44


In [5]:
list = Blackoutdates.values.tolist()

In [7]:
churchesdefinition = pd.read_csv('C:\\Users\\ryder\\Documents\\Software\\ChurchesOptimization\\Churches.csv')

In [11]:
churchesdefinition.set_index('ChurchID')['NumWeeksScheduled'].to_dict()

{0: 4, 1: 2, 2: 3, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4}

In [10]:
churchesdefinition

,ChurchID,ChurchName,NumWeeksScheduled
0,0,FCSalem,4
1,1,Crossroads,2
2,2,St.Peters,3
3,3,Charlton,4
4,4,Mechanicsburg CB,4
5,5,Camp Hill UMC,4
6,6,Grace UMC,4
7,7,Dillsburg CB,4
8,8,Our Savior,4
